In [16]:
import pickle
import os
import tarfile

import nltk
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import requests
from nltk.translate import bleu_score
from sklearn.model_selection import train_test_split
from tqdm import tqdm

from model import Encoder, Decoder, NMT
from vocab import Vocab

torch.set_default_tensor_type(torch.cuda.FloatTensor)

# Local Attention in Neural Machine Translation

A pytorch implementation of ["Effective Approaches to Attention-based Neural Machine Translation" by Luong, Pham, and Manning (2015).](https://arxiv.org/abs/1508.04025)

The first three sections of this text will be exposition, and can be skimmed over or skipped entirely if you'd like to go straight to the methods and/or results section.

##### Foreward

##### Introduction
  - Languages
  - Machine Translation

##### Background
  - Seq2Seq
  - Neural Machine Translation
  - Attention Mechanisms

##### Methods
  - LSTMs (seq2seq)
  - Attention Mechanisms
    - global
    - local
  - Europarl (English-Spanish)

##### Results
  - Vanilla seq2seq
  - Attention
    - global
    - local

##### Conclusion


## Foreward

> *If you think a good translation is expensive, you should look at the cost of a bad translation*

I'm not much of a language translator myself, though I like to think of myself as a pretty good translator of mathematics and physics. In the domain of natural languages, I just know conversational Spanish, however probably learned too late in life to ever hope to translate with any effectiveness. Depending on the how and when, learning languages is both the most effortless and one of the most difficult tasks for humans. 

Translation between languages is a notoriously difficult task, even for humans. If you are an avid reader, you may have come across texts translated from another language. Frequently, if the work is old and popular enough, there can be much deliberation over which translation does the original work the most justice. In the abstract we can think of language as semantic descriptions of the world, even if on a deeper level, they are expressions of human ideas.

The translation of languages using computers starts in the 1930s with translation between English and Russian.

As it is my one extra language, we'll be exploring translation of English phrases into Spanish phrases. We have a nice Spanish phrase to illustrate some of our difficulties in moving forward:


*Es un idioma*, generally translates to in English: *It's a language*

However, it can also be used to mean: *It's an idiom*, a figurative phrase which is difficult to translate.

So it's also used as a dismissal for such difficult to translate phrases, 'I can't really translate that... *Es un idioma*'

I think it's a self-aware symbolism that these Spanish homonyms mean either *idiom* or *language*.

With that in mind, let's get started!

##### Introduction



## Background

The sequence to sequence (seq2seq) architecture is a common RNN pattern which allows for variable length input and output layers. It allows a network to learn both an embedding for an input sequence, and to emit an output sequence from that embedding. These tasks are given to two separate RNN units or networks, which we will call the "encoder" and the "decoder". This architecture is well suited to machine translation tasks, as sentences vary in length within languages and between language translations. A vocabulary is commonly constructed for each language in order for the network to represent the words of each language as a finite collection. First, the input vocabulary will be transformed into its integer representation (or a one-hot encoding), and fed into the encoder.

![encoder-decoder](img/lstm.png)

[figure found online](https://smerity.com/articles/2016/google_nmt_arch.html)

The architecture above displays this process for English-to-German. For each word in the English phrase, the encoder takes the input feature vector, and passes it along via its hidden state, as shown on the left. The task of each time step of the decoder, is to output/emit a layer which is then fed into a softmax activation, as shown on the right. This softmax represents our probabilities or logits in the space of our output vocabulary.

##### Attention

Attention mechanisms in deep learning are an effective method of addressing *vanishing gradients* as well as assisting learning in highly overparameterized or large input models. The name suggests the relationship with human experience, where the network is allowed to *focus* on a subset of a layer, by masking or weighting the contributions of a layer. Some attention mechanisms are stochastic, and non-differentiable. Here we are only concerned with differentiable attention on LSTMs.

An attention mechanism is said to be *global* where it acts as a mask or smooth (i.e. weighted) mask across an entire layer. This addresses the problems stated above, and allows the model to converge. However, it can increase the training time significantly by demanding a large number of (generally trivial) computations. A *local* attention method resolves this problem by somehow choosing which inputs to consider, instead of zeroing out a large number of inputs. The challenge with local methods is to conceive of a *good*, *differentiable* local attention mechanism.


## Methods

##### Encoder-Decoder Network

![lstm](img/lstm_diagram.png)
Luong et al. Figure 1

For our basic network we use the seq2seq model with identical bidirectional LSTM networks for the encoder and decoder, as depicted in the figure above. This figure depicts a forward pass of the model during testing, where the  input which is fed into the decoder each step is the output of the previous step, with the exception of a start token. 

During training, by contrast, the $X', Y', Z'$ predictions of the decoder are used to compute the loss, however the inputs are determined by the ground truth labels. This is especially important for early convergence, where the output sequence will be too chaotic to learn from. Eventually weaning the network off of ground truth sequences as inputs might be a good idea to further improve model results.

##### Global Attention

![global attention](img/lstm_global_attention_diagram.png)
Luong et al. Figure 2.

The figure about depicts the global attention mechanism. The vector $a_t$, the attention vector, is computed from the hidden states across the entirety of the input sequence.

##### Local Attention

![local attention](img/lstm_local_attention_diagram.png)

The figure about depicts the local attention mechanism. The vector $a_t$, the attention vector, is computed from the hidden states across a small window of the input sequence.


The input layer size and hidden layer size were chosen arbitrarily, but in principle can be heuristically optimized using cross-validation methods.

To train the model, we use the [europarl](http://www.statmt.org/europarl/) corpus, a parallel corpus of translated proceedings of the European Parliament from 1996-2011. The corpus contains translations between 21 European languages, and has already (mostly) been preprocessed.

The Spanish/English pairs are downloaded using the code below.

In [2]:
# collect our data
language = 'es'
tarfilename = "{}-en.tgz".format(language)
tarfilepath = os.path.join("data/", tarfilename)
englishfile = 'data/europarl-v7.{}-en.en'.format(language)
spanishfile = 'data/europarl-v7.{0}-en.{0}'.format(language)
def maybe_download():
    if not os.path.exists(tarfilepath):
        print('downloading {}...'.format(tarfilename))
        url = "http://www.statmt.org/europarl/v7/{}".format(tarfilename)
        os.makedirs('data/', exist_ok=True)
        r = requests.get(url, stream=True)
        with open(tarfilepath, 'wb') as fd:
            for content in r.iter_content():
                fd.write(content)
        print('download complete!')
    if not os.path.exists(englishfile):
        print('Extracting...')
        with tarfile.open(tarfilepath) as tar:
            tar.extractall(path='data/')
        print('done!')
        
maybe_download()

In [3]:
# data data data

def build_full_vocabs():
    with open(englishfile) as en_fd, open(spanishfile) as es_fd:
        en_lang = Vocab(name='english')
        es_lang = Vocab(name='spanish')
        try:
            en_lang.add_corpus(en_fd)
        except VocabFull:
            pass
        try:
            es_lang.add_corpus(es_fd)
        except:
            pass
    en_lang.calcify()
    es_lang.calcify()
    return en_lang, es_lang


In [4]:
def corpora2vectors():
    with open(englishfile) as en_fd, open(spanishfile) as es_fd:
        eng = []
        esp = []
        reg = re.compile('[a-zA-Z]')
        for en, es in zip(en_fd, es_fd):
            en = en.strip()
            es = es.strip()
            if not reg.search('[a-zA-Z]', en):
                continue
            if not reg.search('[a-zA-Z]', es):
                continue
            eng.append(en_lang.tokens2tensor(en_lang.word_tokenize(en)))
            esp.append(es_lang.tokens2tensor(es_lang.word_tokenize(es)))
    return eng, es


In [5]:
# don't rebuild corpus
if os.path.exists('X.pkl'):
    with open('en_lang.pkl', 'rb') as fd:
        en_lang = pickle.load(fd)
    with open('es_lang.pkl', 'rb') as fd:
        es_lang = pickle.load(fd)
    #with open('X.pkl', 'rb') as fd:
    #    X = pickle.load(fd)
    #with open('y.pkl', 'rb') as fd:
    #    y = pickle.load(fd)
else:
    en_lang, es_lang = build_full_vocabs()
    X, y = corpora2vectors()

In [6]:
class Encoder(nn.Module):

    def __init__(self, name: str, vocab: Vocab, embedding_size: int,
                 n_hidden: int, lstm_layers: int):
        """
        Basic Encoder Module for neural machine translation
        :param name: module name
        :param vocab: Vocab object, for mapping numerics to words
        :param embedding_size: size of word embedding
        :param n_hidden: number of hidden nodes in each lstm.
        :param lstm_layers: number of lstm layers
        """
        super().__init__()
        self.__name__ = name

        # Saving this so that other parts of the class can re-use it
        self.n_hidden = n_hidden
        self.n_layers = lstm_layers

        # word embeddings:
        self.input_lookup = nn.Embedding(num_embeddings=vocab.size,
                                         embedding_dim=embedding_size)

        self.lstm = nn.LSTM(input_size=embedding_size,
                            hidden_size=self.n_hidden,
                            num_layers=self.n_layers,
                            bidirectional=False)

    def forward(self, input, hidden):
        """
        basic forward pass of encoder, defined for full inputs
        :param input: encoder lstm inputs
        :param hidden: previous lstm hidden state
        :return:
        """
        embedding = self.input_lookup(input)
        output = embedding
        output, hidden = self.lstm(output, hidden)
        return output, hidden

    def init_hidden(self):
        h0 = torch.zeros(self.n_layers, 1, self.n_hidden).cuda()
        c0 = torch.zeros(self.n_layers, 1, self.n_hidden).cuda()
        return h0, c0

class VanillaDecoder(nn.Module):

    def __init__(self, name: str, vocab: Vocab, embedding_size: int,
                 n_hidden: int, lstm_layers: int):
        """
        Decoder Module without an attention mechanism, for comparison.
        See Decoder for parameter help.
        """
        super().__init__()
        self.__name__ = name
        
        # some useful static properties
        n_pt_weights = n_hidden
        self.lut = vocab.tokens2tensor
        self.out_lut = vocab.tensor2tokens
        self.n_hidden = n_hidden
        self.n_layers = lstm_layers

        # embedding:
        self.output_lookup = nn.Embedding(num_embeddings=vocab.size,
                                          embedding_dim=embedding_size)

        # attention module
        #self.p_t_dense = nn.Linear(self.n_hidden, n_pt_weights, bias=False)
        #self.p_t_dot = nn.Linear(n_pt_weights, 1, bias=False)
        #self.score = nn.Bilinear(self.n_hidden, self.n_hidden, 1, bias=False)  # ?

        #self.combine_attention = nn.Linear(2 * self.n_hidden, self.n_hidden)

        self.lstm = nn.LSTM(input_size=embedding_size,  # + self.n_hidden
                            hidden_size=self.n_hidden,
                            num_layers=self.n_layers,
                            bidirectional=False,
                            dropout=0.2)

        self.dense_out = nn.Linear(self.n_hidden, vocab.size)

        self.softmax = nn.LogSoftmax(dim=-1)

    def forward(self, input, hidden, h_s, h_t_tilde):
        embedding = self.output_lookup(input.view(1, 1))
        # context_embedding = torch.cat((embedding, h_t_tilde), dim=-1)

        # lstm
        output, hidden = self.lstm(embedding, hidden)

        y = self.softmax(self.dense_out(output))
        return y, hidden, h_t_tilde, \
            (None, None)  # for conformed interfacing w/ Attention


In [7]:
class Decoder(nn.Module):

    def __init__(self, name: str, vocab: Vocab, embedding_size: int,
                 n_hidden: int, lstm_layers: int, local_window):
        """
        Decoder Module with Attention Mechanism for neural machine translation
        :param name: name for object instance
        :param vocab: output vocabulary for predictions
        :param embedding_size:
        :param n_hidden:
        :param lstm_layers:
        :param local_window:
        """
        super().__init__()
        self.__name__ = name

        n_pt_weights = n_hidden

        # Saving this so that other parts of the class can re-use it
        self.n_hidden = n_hidden
        self.n_layers = lstm_layers
        self.out_lut = vocab.tensor2tokens
        self.local_window = local_window

        # word embeddings:
        self.output_lookup = nn.Embedding(num_embeddings=vocab.size,
                                          embedding_dim=embedding_size)

        # attention module
        self.p_t_dense = nn.Linear(self.n_hidden, n_pt_weights, bias=False)
        self.p_t_dot = nn.Linear(n_pt_weights, 1, bias=False)
        self.score = nn.Bilinear(self.n_hidden, self.n_hidden, 1, bias=False)  # ?

        self.combine_attention = nn.Linear(2 * self.n_hidden, self.n_hidden)

        self.lstm = nn.LSTM(input_size=embedding_size + self.n_hidden,
                            hidden_size=self.n_hidden,
                            num_layers=self.n_layers,
                            bidirectional=False,
                            dropout=0.2)

        self.dense_out = nn.Linear(self.n_hidden, vocab.size)

        self.softmax = nn.LogSoftmax(dim=-1)

    def forward(self, input, hidden, h_s, h_t_tilde):
        embedding = self.output_lookup(input.view(1, 1))

        context_embedding = torch.cat((embedding, h_t_tilde), dim=-1)

        # lstm
        output, hidden = self.lstm(context_embedding, hidden)

        # attention
        if self.local_window:
            # local
            h_t = hidden[0][-1]
            p_t = h_s.size(0) * torch.sigmoid(self.p_t_dot(torch.tanh(self.p_t_dense(h_t)))).squeeze()  # (9)
            s = torch.round(p_t).long().cuda()
            D = self.local_window
            minimum, maximum = max(s - D, 0), min(s + D, h_s.size(0) - 1)
            h_s_local = h_s[minimum:maximum + 1]  # @@@@@ zero pad? @@@@@
            h_t_rep = h_t.repeat(h_s_local.size(0), 1, 1)
            score = self.score(h_t_rep, h_s_local)  # (8) (general)
            gauss_window = torch.exp((torch.arange(minimum, maximum + 1).float() - p_t) ** 2 / (D / 2) ** 2).view(-1, 1, 1).cuda()
            a_t = torch.softmax(score, dim=0) * gauss_window  # (7) & (10)
            context = torch.mean(a_t * h_s_local, dim=0, keepdim=True)
        else:
            # global
            pass

        h_t_tilde = torch.tanh(self.combine_attention(torch.cat((context, h_t.view(1, 1, -1)), dim=-1)))  # (5)
        y = self.softmax(self.dense_out(h_t_tilde))  # (6)

        return y, hidden, h_t_tilde, (a_t, s)

In [13]:
class NMT:

    def __init__(self, name, encoder, decoder, loss_fn):
        """
        simple wrapper class for handling encoder-decoder structure.
        :param name: Name for NMT instance
        :param encoder: Encoder module instance
        :param decoder: Decoder module instance (must match hidden state size of encoder)
        :param loss_fn: a loss function for comparison with log-softmax layer. Probably NLLLoss(dim=-1)
        """
        self.__name__ = name
        self.encoder = encoder
        self.decoder = decoder
        self.loss_fn = loss_fn
        self.optimizer = optim.Adam((*encoder.parameters(), *decoder.parameters()), lr=1e-3)
        self.epochs = 0

    def set_optimizer(self, optimizer, learning_rate=1e-3):
        """
        explicitly set the optimizer
        :param optimizer: new optimizer
        :param learning_rate: new learning rate
        :return:
        """
        self.optimizer = optimizer((*self.encoder.parameters(),
                                    *self.decoder.parameters()),
                                   lr=learning_rate)

    def train(self, X, y, epochs=1, batch_size=1, clipping=0.25, print_every=4, examples=[],
              examples_epoch_fn=lambda x: True):
        """
        epoch training function
        :param X: input training data
        :param y: output training data
        :param epochs: number of epochs to train
        :param batch_size: batch size (WARNING: Empirically doesn't work)
        :param clipping: gradient clipping coef
        :param print_every: print loss at every N% completion of each epoch
        :param examples: optional training examples for displaying progress.
        :param examples_epoch_fn: function which takes the epoch number, and returns a boolean value.
        Training examples are only printed for epochs where the function returns true.
        e.g. lambda x: x**(1/2) == int(x**(1/2))
        :return: None
        """
        N = len(X)

        for e in range(epochs):
            print_loss_total = prev_log_iter = prev_log_percent = 0
            for n in range(N):
                xi = X[n]
                yi = y[n]
                step = True
                print_loss_total += self._train(xi.cuda(), yi.cuda(), clipping, step=step, loss_factor=batch_size) * batch_size
                if step and 100 * n // N >= prev_log_percent + print_every:
                    print_loss_avg = print_loss_total / (n - prev_log_iter)
                    prev_log_iter = n
                    prev_log_percent = 100 * n // N
                    print('\repoch %s: %%%s complete     avg loss: %.4f' %
                          (self.epochs, prev_log_percent, print_loss_avg))
                    print_loss_total = 0
            self.epochs += 1
            if examples_epoch_fn(e):
                for ex in examples:
                    output = self.predict(ex)
                    print(' '.join(self.decoder.out_lut(output)))

    def predict(self, input, cap: int = 20):
        """
        returns prediction vector from input
        :param input: input statement
        :param cap: max length for prediction.
        :return:
        """
        hidden = self.encoder.init_hidden()
        encoded, hidden = self.encoder(input.view(-1, 1), hidden)
        inp = torch.tensor([[1]]).cuda()  # EOS_TOKEN
        h_t_tilde = hidden[0][-1].unsqueeze(0) * 0
        output = []
        for di in range(cap):
            logits, hidden, h_t_tilde, decoder_attention = self.decoder(inp, hidden, encoded, h_t_tilde)
            topv, topi = logits.topk(1)
            inp = topi.squeeze().detach()  # detach from history as input
            if inp.item() == 1:  # EOS_TOKEN
                break
            output.append(inp.squeeze())

        return torch.stack(tuple(output), dim=0)

    def save(self, path=None):
        if not path:
            path = os.path.join('.', self.name)
        encoder = path + '_encoder'
        decoder = path + '_decoder'
        torch.save(self.encoder.state_dict(), encoder)
        torch.save(self.decoder.state_dict(), decoder)

    def load(self, path):
        encoder = path + '_encoder'
        decoder = path + '_decoder'
        enc = torch.load(encoder)
        dec = torch.load(decoder)
        self.encoder.load_state_dict(enc)
        self.decoder.load_state_dict(dec)
        
    def _train(self, x, y, clipping, step, loss_factor=1):
        """
        single example training
        :param x: single input sentence vector
        :param y: single output sentence vector
        :param clipping: gradient clipping
        :param step: execute optimizer step or not
        :param loss_factor: basic factor for batch averaging.
        :return:
        """
        hidden = self.encoder.init_hidden()

        target_length = y.size(0)

        loss = 0
        x = x.view(-1, 1)
        h_s, hidden = self.encoder(x, hidden)

        bos = torch.tensor(
            [[0]]).cuda()  # BOS_TOKEN
        inp = bos
        h_t_tilde = hidden[0][-1].unsqueeze(0) * 0
        for di in range(target_length):
            logits, hidden, h_t_tilde, decoder_attention = self.decoder(inp, hidden, h_s, h_t_tilde)
            topv, topi = logits.topk(1)
            inp = topi.squeeze().detach()  # detach from history as input
            loss += self.loss_fn(logits.view(1, -1), y[di].view(1)) / loss_factor
            if inp.item() == 1:  # EOS_TOKEN
                break
            inp = y[di]

        loss.backward()
        nn.utils.clip_grad_norm_((*self.encoder.parameters(), *self.decoder.parameters()), clipping)

        if step:
            self.optimizer.step()
            self.optimizer.zero_grad()

        return loss.item() / target_length

In [9]:
encoder = Encoder('encoder', vocab=en_lang, embedding_size=100, n_hidden=70, lstm_layers=2)
decoder = Decoder('decoder', vocab=es_lang, embedding_size=100, n_hidden=70, lstm_layers=2, local_window=2)

In [14]:
nmt = NMT('local_nmt', encoder, decoder, nn.NLLLoss())

In [21]:
# nmt.train(X_train, y_train, epochs=100, batch_size=1, examples=X_test[:5])
nmt.load('nmt_local_2')

In [17]:
with open('X_test.pkl', 'rb') as fd:
    X_test = pickle.load(fd)
with open('y_test.pkl', 'rb') as fd:
    y_test = pickle.load(fd)

In [23]:
for i, (en, es) in enumerate(zip(X_test, y_test)):
    en_sent = en_lang.tensor2tokens(en)
    es_sent = decoder.out_lut(es)
    pred_es = nmt.predict(en)
    pred_es_sent = decoder.out_lut(pred_es)
    if i > 5:
        break
    print("in:    %s\ntrans: %s\ntrue:  %s\n" % (' '.join(en_sent), ' '.join(pred_es_sent), ' '.join(es_sent)))


in:    doctors dispensaries and social support structures are needed
trans: que la comisión ha sido el sr prodi y que la comisión ha sido el sr prodi y la comisión
true:  se necesitan médicos ambulatorios estructuras sociales complementarias

in:    this would show respect for these peripheral areas which are rather concerned about enlargement to the east
trans: que la comisión ha sido en el parlamento europeo y la comisión ha sido un informe de la comisión y
true:  ello sería dar muestra de respeto hacia esas zonas periféricas algo inquietas con la ampliación hacia el este

in:    europe must i feel today establish a genuine codevelopment policy related to migratory flows which should mr president centre on a few simple proposals such as the framing of aid policies for projects by migrants devising instruments for channelling the savings of immigrants the placement of students in training and the placement of young trainees
trans: que la comisión ha sido un instrumento que se ha prese

In [ ]:
scores = []
for i, (en, es) in enumerate(zip(X_test, y_test)):
    # en_sent = en_lang.tensor2tokens(en)
    es_sent = decoder.out_lut(es)
    pred_es = nmt.predict(en)
    pred_es_sent = decoder.out_lut(pred_es)
    scores.append(bleu_score.sentence_bleu(pred_es_sent, es_sent))

/home/euler/anaconda3/envs/nlp/lib/python3.7/site-packages/nltk/translate/bleu_score.py:523: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/euler/anaconda3/envs/nlp/lib/python3.7/site-packages/nltk/translate/bleu_score.py:523: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/euler/anaconda3/envs/nlp/lib/python3.7/site-packages/nltk/translate/bleu_score.py:523: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower 

In [27]:
np.mean(scores)